# Kaggle - TMDB

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score

In [4]:
train['coll_id'] = train['belongs_to_collection'].fillna('[{"id":0}]').map(lambda x: (ast.literal_eval(x))[0]['id'])

Dictionaries taken from `tmdb-extract-tables-log`

In [5]:
l_short = {'en': 'en',
 'fr': 'fr',
 'ru': 'ru',
 'es': 'es',
 'hi': 'hi',
 'ja': 'ja',
 'it': 'it',
 'ko': 'ko',
 'cn': 'cn',
 'zh': 'zh',
 'de': 'de',
 'ta': 'ta'}

In [6]:
l_rev = {'en': 16.201695631547217,
 'fr': 13.718204235553607,
 'ru': 13.815132182879807,
 'es': 14.645970166012837,
 'hi': 15.371121660763546,
 'ja': 15.818050019285394,
 'it': 14.610307296701814,
 'ko': 14.561503498231747,
 'cn': 15.720496475312752,
 'zh': 15.246036823468886,
 'de': 14.583008872938295,
 'ta': 15.073328869838628}

In [9]:
train['l_short'] = train['original_language'].map(l_short).fillna('other')

In [10]:
train['l_rev'] = train['original_language'].map(l_rev).fillna(13.61844005781211)

In [5]:
colls = pd.read_csv('collections.csv',index_col=0)
colls.head()

,average_revenue,stdev
10,749699164,141846802
84,531269279,232145436
119,898827882,38833622
151,96733333,32511741
230,190916236,76579911


In [6]:
train['coll_rev'] = train['coll_id'].map(colls['average_revenue']).fillna(0)

In [7]:
reg = LinearRegression()
ml = MLPRegressor()
boost = GradientBoostingRegressor()

In [40]:
# previously established that coll_rev is a good feature
X = train[['budget','popularity','coll_rev']].values
y = train['revenue'].values

In [9]:
cross_val_score(reg,X,y,cv=10).mean()

0.7616451754160671

In [10]:
cross_val_score(ml,X,y,cv=10).mean()

0.7640482590479235

In [11]:
cross_val_score(boost,X,y,cv=10).mean()

0.7520894559506123

### Add genres

In [12]:
genres = pd.read_csv('genres.csv',index_col=0)
genres.head()

,name,revenue
10752,War,4.991587e+07
10402,Music,4.287090e+07
35,Comedy,6.087574e+07
99,Documentary,4.638009e+06
37,Western,5.137064e+07


**As dummy variables**

In [13]:
genres.index

Int64Index([10752, 10402,    35,    99,    37,    36,    12,   878,    14,
               16,    80,    18,  9648, 10769,    53, 10770,    27,    28,
            10749, 10751],
           dtype='int64')

In [14]:
train['genres_id'] = train['genres'].fillna('[{"id":-1}]').map(lambda x: [i['id'] for i in (ast.literal_eval(x))])

In [15]:
# test with a few
g = 28
name = f'genre_{g}'
train['genres_id'].head(10).map(lambda x: 1 if g in x else 0)

0    0
1    0
2    0
3    0
4    1
5    0
6    0
7    0
8    1
9    0
Name: genres_id, dtype: int64

In [16]:
# build dummy variables
for g in genres.index:
    name = f'genre_{g}'
    train[name] = train['genres_id'].map(lambda x: 1 if g in x else 0)

In [17]:
train['genres_id'].head()

0                      [35]
1    [35, 18, 10751, 10749]
2                      [18]
3                  [53, 18]
4                  [28, 53]
Name: genres_id, dtype: object

In [18]:
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,genre_80,genre_18,genre_9648,genre_10769,genre_53,genre_10770,genre_27,genre_28,genre_10749,genre_10751
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,0,0,0,0,0,0,0,0,0,0
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,0,1,0,0,0,0,0,0,1,1
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,0,1,0,0,0,0,0,0,0,0
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,0,1,0,0,1,0,0,0,0,0
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,0,0,0,0,1,0,0,1,0,0


Looks correct

In [19]:
cols_to_consider = ['budget','popularity','coll_rev']+[f'genre_{g}' for g in genres.index]

In [20]:
X_genre = train[cols_to_consider].values

In [21]:
#cross_val_score(reg,X_genre,y,cv=10).mean(),cross_val_score(ml,X_genre,y,cv=10).mean(),cross_val_score(boost,X_genre,y,cv=10).mean()

Marginal improvement

**As mean of revenues**

In [22]:
train['genres_id'].head()

0                      [35]
1    [35, 18, 10751, 10749]
2                      [18]
3                  [53, 18]
4                  [28, 53]
Name: genres_id, dtype: object

In [23]:
gen_rev = dict(genres['revenue'])
gen_rev[35]

60875742.9192607

In [24]:
# there was an issue with some movies that don't have any genre

In [25]:
null_mean = train[train['genres'].isnull()]['revenue'].mean()

In [26]:
gen_rev[-1] = null_mean

In [27]:
train['genre_ave'] = train['genres_id'].map(lambda x: np.array([gen_rev[g] for g in x]).mean())
train['genre_top'] = train['genres_id'].map(lambda x: np.array([gen_rev[g] for g in x]).max())

In [28]:
# try either

In [29]:
X_ave  = train[['budget','popularity','coll_rev','genre_ave']].values
X_top  = train[['budget','popularity','coll_rev','genre_top']].values
X_both = train[['budget','popularity','coll_rev','genre_ave','genre_top']].values

In [30]:
def consider_all(X):
    a = cross_val_score(reg,X,y,cv=10).mean()
    b = cross_val_score(ml,X,y,cv=10).mean()
    c = cross_val_score(boost,X,y,cv=10).mean()
    return (a,b,c)

In [31]:
#consider_all(X_ave)

In [32]:
#consider_all(X_top)

In [33]:
#consider_all(X_both)

It appears that dummy variables work best in this case<br>
Although the score is not much better, I will leave it in for now

### Test set

In [34]:
# collection
test['coll_id'] = test['belongs_to_collection'].fillna('[{"id":0}]').map(
    lambda x: (ast.literal_eval(x))[0]['id'])
test['coll_rev'] = test['coll_id'].map(colls['average_revenue']).fillna(0)
# genres
test['genres_id'] = test['genres'].fillna('[{"id":-1}]').map(
    lambda x: [i['id'] for i in (ast.literal_eval(x))])
for g in genres.index:
    name = f'genre_{g}'
    test[name] = test['genres_id'].map(lambda x: 1 if g in x else 0)

In [35]:
test['genre_ave'] = test['genres_id'].map(lambda x: np.array([gen_rev[g] for g in x]).mean())
test['genre_top'] = test['genres_id'].map(lambda x: np.array([gen_rev[g] for g in x]).max())

In [36]:
T_ave  = test[['budget','popularity','coll_rev','genre_ave']].values
T_top  = test[['budget','popularity','coll_rev','genre_top']].values
T_both = test[['budget','popularity','coll_rev','genre_ave','genre_top']].values

In [37]:
submit = pd.read_csv('sample_submission.csv',index_col='id')
submit.head()

,revenue
id,
3001,1000000
3002,1000000
3003,1000000
3004,1000000
3005,1000000


In [41]:
y = np.log(y)
boost.fit(X_ave,y)
pred = boost.predict(T_ave)

#submit['revenue'] = pred
#submit.to_csv('log_ave_gen.csv')

In [50]:
pred = np.exp(pred)

In [51]:
submit['revenue'] = pred
submit.to_csv('log_ave_gen.csv')
### note that the avereages are not logged

In [48]:
np.log(12314651)

16.32630024971174

In [49]:
np.exp(y[0])

12314651.000000002